# TMDB 5000 Movie Dataset으로 장르기반 추천시스템 만들기
* 장르에 있는 텍스트를 벡터화 후 텍스트간 유사도를 구해 가까운 순서대로 정렬
* 유사도 척도: 코사인유사도

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

ModuleNotFoundError: No module named 'koreanize_matplotlib'

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv")
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [4]:
data['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [5]:
import json

In [6]:
json.loads(data['genres'][0])[0]['name']

'Action'

In [7]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [8]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [9]:
dict_cols = ['genres','keywords', 'production_companies','production_countries','spoken_languages']
dict_cols

['genres',
 'keywords',
 'production_companies',
 'production_countries',
 'spoken_languages']

In [10]:
def dict_unpack(data):
    for col in dict_cols:
        temp = []
        for i in json.loads(data[col]):
            temp.append(i.get('name','None'))
        #print(temp)
        temp = "".join(temp)
#        print(temp)
        data[col] = temp
    return data

In [11]:
data = data.apply(dict_unpack,axis=1)

In [12]:
data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,ActionAdventureFantasyScience Fiction,http://www.avatarmovie.com/,19995,culture clashfuturespace warspace colonysociet...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film PartnersTwentieth Century Fox F...,United States of AmericaUnited Kingdom,2009-12-10,2787965087,162.0,EnglishEspañol,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,AdventureFantasyAction,http://disney.go.com/disneypictures/pirates/,285,oceandrug abuseexotic islandeast india trading...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney PicturesJerry Bruckheimer FilmsSec...,United States of America,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,ActionAdventureCrime,http://www.sonypictures.com/movies/spectre/,206647,spybased on novelsecret agentsequelmi6british ...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia PicturesDanjaqB24,United KingdomUnited States of America,2015-10-26,880674609,148.0,FrançaisEnglishEspañolItalianoDeutsch,Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,ActionCrimeDramaThriller,http://www.thedarkknightrises.com/,49026,dc comicscrime fighterterroristsecret identity...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary PicturesWarner Bros.DC Entertainment...,United States of America,2012-07-16,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,ActionAdventureScience Fiction,http://movies.disney.com/john-carter,49529,based on novelmarsmedallionspace travelprinces...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,United States of America,2012-03-07,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,ActionCrimeThriller,NaN,9367,united states–mexico barrierlegsarmspaper knif...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,Columbia Pictures,MexicoUnited States of America,1992-09-04,2040920,81.0,Español,Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,ComedyRomance,NaN,72766,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,,,2011-12-26,0,85.0,,Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,ComedyDramaRomanceTV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,datelove at first sightnarrationinvestigationt...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,Front Street PicturesMuse Entertainment Enterp...,United States of America,2013-10-13,0,120.0,English,Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,,http://shanghaicalling.com/,126186,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,,United States of AmericaChina,2012-05-03,0,98.0,English,Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7


In [13]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [14]:
data = data[['original_title','genres','popularity','runtime', 'vote_average','vote_count']]
data

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,ActionAdventureFantasyScience Fiction,150.437577,162.0,7.2,11800
1,Pirates of the Caribbean: At World's End,AdventureFantasyAction,139.082615,169.0,6.9,4500
2,Spectre,ActionAdventureCrime,107.376788,148.0,6.3,4466
3,The Dark Knight Rises,ActionCrimeDramaThriller,112.312950,165.0,7.6,9106
4,John Carter,ActionAdventureScience Fiction,43.926995,132.0,6.1,2124
...,...,...,...,...,...,...
4798,El Mariachi,ActionCrimeThriller,14.269792,81.0,6.6,238
4799,Newlyweds,ComedyRomance,0.642552,85.0,5.9,5
4800,"Signed, Sealed, Delivered",ComedyDramaRomanceTV Movie,1.444476,120.0,7.0,6
4801,Shanghai Calling,,0.857008,98.0,5.7,7


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4803 non-null   object 
 1   genres          4803 non-null   object 
 2   popularity      4803 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4803 non-null   float64
 5   vote_count      4803 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 225.3+ KB


In [16]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4801 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4801 non-null   object 
 1   genres          4801 non-null   object 
 2   popularity      4801 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4801 non-null   float64
 5   vote_count      4801 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 262.6+ KB


# 장르기반 추천시스템 만들기
* 장르 컬럼 텍스트를 숫자로 벡터화
* 코사인 유사도를 구해 비슷한 장르의 행 검색
* 유사도가 높은 행을 기준으로 내림차순 정렬

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [58]:
cv = CountVectorizer(ngram_range=(1,2))
genres_mat = cv.fit_transform(data['genres'])
print(len(cv.get_feature_names_out()))
genres_mat_df = pd.DataFrame(genres_mat)
genres_mat_df

1394


,0
0,"(np.int32(0), np.int32(77))\t1\n (np.int32(..."
1,"(np.int32(0), np.int32(386))\t1"
2,"(np.int32(0), np.int32(41))\t1"
3,"(np.int32(0), np.int32(160))\t1"
4,"(np.int32(0), np.int32(1010))\t1\n (np.int3..."
...,...
4796,"(np.int32(0), np.int32(167))\t1"
4797,"(np.int32(0), np.int32(619))\t1"
4798,"(np.int32(0), np.int32(1145))\t1\n (np.int3..."
4799,


In [59]:
genres_mat_df.loc[0,:]

0      (np.int32(0), np.int32(77))\t1\n  (np.int32(...
Name: 0, dtype: object

* 코사인 유사도를 이용해 genners_mat의 유사도 산출 후 genres_sim 생성
* 비슷한 장르의 영화를 찾기 위해서 계산
* 코사인 유사도는 주로 문자,문서의 유사도를 계산하는데 사용 됨

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

In [61]:
genres_sim = cosine_similarity(genres_mat,genres_mat)
print(genres_sim.shape)

(4801, 4801)


In [62]:
genres_sim_df = pd.DataFrame(genres_sim)
genres_sim_df

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,1.000000,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.333333,0.0,0.0,0.0,1.000000,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4796,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4797,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4798,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4799,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
sorted_genres_sim = genres_sim.argsort()[:,::-1]

In [49]:
sorted_genres_sim[:1]

array([[   0, 3493,  870, ...,    2,    1,   24]])

In [50]:
data.loc[14,:]

original_title                             Man of Steel
genres            ActionAdventureFantasyScience Fiction
popularity                                    99.398009
runtime                                           143.0
vote_average                                        6.5
vote_count                                         6359
Name: 14, dtype: object

In [51]:
data.loc[3493,:]

original_title    Morvern Callar
genres                     Drama
popularity              2.507912
runtime                     97.0
vote_average                 7.2
vote_count                    34
Name: 3493, dtype: object

In [52]:
data.loc[46,:]

original_title               X-Men: Days of Future Past
genres            ActionAdventureFantasyScience Fiction
popularity                                   118.078691
runtime                                           131.0
vote_average                                        7.5
vote_count                                         6032
Name: 46, dtype: object

# 영화 이름과 추천 받을 개수를 입력받아 추천영화 출력하기

In [53]:
movie_index = data[data['original_title'] == 'Avatar'].index.values[0]

In [54]:
recommaned_idx = sorted_genres_sim[movie_index,1:11]

In [55]:
data.loc[recommaned_idx,:]

,original_title,genres,popularity,runtime,vote_average,vote_count
3493,Morvern Callar,Drama,2.507912,97.0,7.2,34
870,Superman II,ActionAdventureFantasyScience Fiction,30.515175,127.0,6.5,629
813,Superman,ActionAdventureFantasyScience Fiction,48.507081,143.0,6.9,1022
46,X-Men: Days of Future Past,ActionAdventureFantasyScience Fiction,118.078691,131.0,7.5,6032
14,Man of Steel,ActionAdventureFantasyScience Fiction,99.398009,143.0,6.5,6359
4174,El Crimen del Padre Amaro,DramaRomance,2.179428,118.0,6.2,36
4464,Død snø,HorrorComedy,11.205726,91.0,6.1,311
3928,Willy Wonka & the Chocolate Factory,FamilyFantasy,23.981601,100.0,7.4,798
2816,Ace Ventura: Pet Detective,ComedyMystery,41.249004,86.0,6.4,1648
2994,Recess: School's Out,Science FictionAnimationComedyFamily,5.699058,83.0,6.6,67


In [56]:
def find_sim_movie(data,sorted_genres_sim,title_name,top_n=10):
    title_movie = data[data['original_title']==title_name]
    title_index =  title_movie.index.values[0]
    similer_idxs = sorted_genres_sim[title_index,1:top_n+1]
    return data.iloc[similer_idxs]

In [57]:
find_sim_movie(data,sorted_genres_sim,"Vessel",20)

,original_title,genres,popularity,runtime,vote_average,vote_count
4737,"Peace, Propaganda & the Promised Land",Documentary,0.020168,80.0,6.4,4
4009,2016: Obama's America,Documentary,0.973582,87.0,4.6,19
4491,The Hadza: Last of the First,Documentary,0.045648,70.0,0.0,0
3503,Lake of Fire,Documentary,0.475748,152.0,8.0,10
4007,"Fat, Sick & Nearly Dead",Documentary,0.545825,97.0,7.1,47
3446,Fahrenheit 9/11,Documentary,16.209850,122.0,6.8,397
4731,Indie Game: The Movie,Documentary,7.676677,96.0,7.4,203
3891,Bowling for Columbine,Documentary,11.414843,120.0,7.3,453
3979,Freakonomics,Documentary,1.781462,93.0,6.6,36
4387,A LEGO Brickumentary,Documentary,1.631274,93.0,6.4,55
